This script will prepare a new spatial database to hold the KSP, KYTC API and WeatherService data.


In [32]:
import os
import pandas as pd
import sqlite3

In [72]:
# Define the path for the SQLite database
database_path = r'/Users/terid/Git/CodeYou_Capstone/data/crash_data.db'
os.makedirs(os.path.dirname(database_path), exist_ok=True)

# Define the path for the raw crash data files downloaded
directory_path = '/Users/terid/Git/CodeYou_Capstone/data/raw_crash_data'

In [34]:
# Function to check if tables created in the database exist
def check_table_exists(db_path, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Query to check if the table exists
    cursor.execute('''
        SELECT name
        FROM sqlite_master
        WHERE type='table' AND name=?
    ''', (table_name,))

    # Fetch one record
    table_exists = cursor.fetchone() is not None

    # Close the connection
    conn.close()

    return table_exists

In [35]:
# Function to check if tables created in the database have data
def check_table_has_data(db_path, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Query to count the number of rows in the table
    cursor.execute(f'SELECT COUNT(*) FROM {table_name}')

    # Fetch the count
    row_count = cursor.fetchone()[0]

    # Close the connection
    conn.close()

    return row_count > 0


In [73]:
# Create/Connect to SQLite database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

In [63]:
# Create collision incidents table in database
cursor.execute('''CREATE TABLE IF NOT EXISTS collision_incidents (
        IncidentID int,
        AgencyORI int,
        AgencyName TEXT,
        IncidentStatusDesc TEXT,
        County TEXT,
        RdwyNumber TEXT,
        Street TEXT,
        RoadwayName TEXT,
        StreetSfx TEXT,
        StreetDir TEXT,
        IntersectionRdwy TEXT,
        IntersectionRdwyName TEXT,
        BetweenStRdwy1 TEXT,
        BetweenStRdwyName1 TEXT,
        BetweenStRdwy2 TEXT,
        BetweenStRdwyName2 TEXT,
        Latitude REAL,
        Longitude REAL,
        Milepoint REAL,
        CollisionDate DATE,
        CollisionTime TIME,
        UnitsInvolved INT,
        MotorVehiclesInvolved INT,
        NumberKilled INT,
        NumberInjured INT,
        Weather TEXT,
        RdwyConditionCode INT,
        HitandRun TEXT,
        DirAnalysisCode	TEXT,
        MannerofCollision TEXT,
        RdwyCharacter TEXT,
        LightCondition TEXT,
        RampFromRdwyId TEXT,
        RampToRdwyId TEXT,
        AcceptedDate DATE,
        IsSecondaryCollision TEXT,
        OwnerBadge TEXT,
        IncidentStatus TEXT);''')

In [38]:
# Create table incident_traffic_control in database
cursor.execute('''CREATE TABLE IF NOT EXISTS incident_traffic_control (
        IncidentID int,
        TrafficControlNo int,
        TrafficControl TEXT);''')

In [39]:
# Create table incident_vehicles in database
cursor.execute('''CREATE TABLE IF NOT EXISTS incident_vehicles (
        IncidentID INT,
        UnitNumber INT,
        UnitType TEXT,
        AirbagSwitchCde TEXT,
        IsCommercialVeh TEXT,
        CrashAvoidCde TEXT,
        DriverIdentifiedCde TEXT,
        EventCollWithFirstCde TEXT,
        EventCollWithSecondCde TEXT,
        HasFire TEXT,
        PreCollActionCde TEXT,
        UnderOverrideCde TEXT,
        VehicleIsInsured TEXT,
        MakeCde TEXT,
        ModelCde TEXT,
        VehicleType TEXT,
        MakeDescription TEXT,
        ModelDescription TEXT);''')

In [40]:
# temp section to read the column names to create table columns

def read_column_names(csv_file_path):

    # Read only the first row of the CSV to get the column names
    df = pd.read_csv(csv_file_path, nrows=0)
    column_names = df.columns.tolist()
    return column_names

# Example usage:
#csv_file_path = '/Users/terid/Git/CodeYou_Capstone/data/raw_crash_data/incidents_2024.csv'
#csv_file_path = '/Users/terid/Git/CodeYou_Capstone/data/raw_crash_data/incidentTrafficControl_2024.csv'
csv_file_path = '/Users/terid/Git/CodeYou_Capstone/data/raw_crash_data/Vehicles_2024.csv'

columns = read_column_names(csv_file_path)
print(columns)


['IncidentID', 'UnitNumber', 'UnitType', 'AirbagSwitchCde', 'IsCommercialVeh', 'CrashAvoidCde', 'DriverIdentifiedCde', 'EventCollWithFirstCde', 'EventCollWithSecondCde', 'HasFire', 'PreCollActionCde', 'UnderOverrideCde', 'VehicleIsInsured', 'MakeCde', 'ModelCde', 'VehicleType', 'MakeDescription', 'ModelDescription', 'Unnamed: 18']


In [74]:
# Check to see if tables created exist
table_names = ('collision_incidents','incident_traffic_control','incident_vehicles')
for table in table_names:
    if check_table_exists(database_path, table):
        print(f"The table '{table}' exists.")
    else:
        print(f"The table '{table}' does not exist.")


The table 'collision_incidents' exists.
The table 'incident_traffic_control' exists.
The table 'incident_vehicles' exists.


In [75]:
# Check of tables created are empty

table_names = ('collision_incidents','incident_traffic_control','incident_vehicles')
for table in table_names:
    if check_table_has_data(database_path, table):
        print(f"The table '{table}' contains data.")
    else:
        print(f"The table '{table}' is empty.")


The table 'collision_incidents' contains data.
The table 'incident_traffic_control' is empty.
The table 'incident_vehicles' contains data.


In [68]:

# Append collision_incidents to database table
csv_files = [f for f in os.listdir(directory_path) if f.startswith("Incidents_") and f.endswith(".csv")]

# Initialize an empty list to hold dataframes
dataframes = []

# Iterate through the CSV files and load them into dataframes
for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all dataframes into a single dataframe
combined_incidents_df = pd.concat(dataframes, ignore_index=True)

# Drop the 'Unnamed: 38' column if it exists
if 'Unnamed: 38' in combined_incidents_df.columns:
    combined_incidents_df = combined_incidents_df.drop(columns=['Unnamed: 38'])

# Set the option to display all columns
pd.set_option('display.max_columns', None)

print("All CSV files have been successfully loaded into a single DataFrame.")
print(combined_incidents_df)

All CSV files have been successfully loaded into a single DataFrame.
      IncidentID AgencyORI                     AgencyName IncidentStatusDesc  \
0       32655798    150000   BULLITT COUNTY SHERIFF DEPT.           Accepted   
1       32660760    568000   LOUISVILLE METRO POLICE DEPT           Accepted   
2       32660920    568000   LOUISVILLE METRO POLICE DEPT           Accepted   
3       32646189    150200  LEBANON JUNCTION POLICE DEPT.           Accepted   
4       32654634    930400     OLDHAM COUNTY POLICE DEPT.           Accepted   
...          ...       ...                            ...                ...   
4100    32686833   0930400     OLDHAM COUNTY POLICE DEPT.           Accepted   
4101    32672935   0150000   BULLITT COUNTY SHERIFF DEPT.           Accepted   
4102    32676655   0790000  MARSHALL COUNTY SHERIFF DEPT.           Accepted   
4103    32658708   0150000   BULLITT COUNTY SHERIFF DEPT.           Accepted   
4104    32659228   0150000   BULLITT COUNTY SHERIFF

In [76]:

# Combine all CSV files in the directory that begin with "IncidentTrafficControl_"
#csv_files = [f for f in os.listdir(directory_path) if f.startswith("Incidents_") and f.endswith(".csv")]
csv_files = [f for f in os.listdir(directory_path) if f.startswith("IncidentTrafficControl_") and f.endswith(".csv")]
#csv_files = [f for f in os.listdir(directory_path)  if f.startswith("Vehicles_") and f.endswith(".csv")]



# Initialize an empty list to hold dataframes
dataframes = []

# Iterate through the CSV files and load them into dataframes
for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all dataframes into a single dataframe
#combined_incidents_df = pd.concat(dataframes, ignore_index=True)
combined_incidents_traffic_control_df = pd.concat(dataframes, ignore_index=True)
# Drop the 'Unnamed: 3' column if it exists
if 'Unnamed: 3' in combined_incidents_traffic_control_df.columns:
    combined_incidents_traffic_control_df = combined_incidents_traffic_control_df.drop(columns=['Unnamed: 3'])

# Set the option to display all columns
pd.set_option('display.max_columns', None)

print("All CSV files have been successfully loaded into a single DataFrame.")
print(combined_incidents_traffic_control_df)

All CSV files have been successfully loaded into a single DataFrame.
      IncidentId  TrafficControlNo       TrafficControl
0       26133281                 1     STOP & GO SIGNAL
1       26146419                 1  ADVISORY SPEED SIGN
2       26146419                 2          CENTER LINE
3       26146419                 3                OTHER
4       26146419                 4        WARNING SIGNS
...          ...               ...                  ...
6741    33431439                 3                OTHER
6742    33431439                 4        WARNING SIGNS
6743    33433119                 1               MEDIAN
6744    33433119                 2  ADVISORY SPEED SIGN
6745    33433119                 3        WARNING SIGNS

[6746 rows x 3 columns]


In [78]:

# Specify the path where you want to save the CSV file
#output_path = '/Users/terid/Git/CodeYou_Capstone/data/clean_crash_data/collision_incidents.csv'
output_path = '/Users/terid/Git/CodeYou_Capstone/data/clean_crash_data/incident_traffic_controls.csv'
#output_path = '/Users/terid/Git/CodeYou_Capstone/data/clean_crash_data/incident_vehicles.csv'
# Export the dataframe to a CSV file
#combined_incidents_df.to_csv(output_path, index=False)
combined_incidents_traffic_control_df.to_csv(output_path, index=False)
#vehicles_df.to_csv(output_path, index=False)
print(f"Dataframe exported successfully to {output_path}")


Dataframe exported successfully to /Users/terid/Git/CodeYou_Capstone/data/clean_crash_data/incident_traffic_controls.csv


In [ ]:

# Prepare the SQL insert statement dynamically based on DataFrame columns
columns = ', '.join([f'"{col}"' for col in control_df.columns])

placeholders = ', '.join(['?'] * len(control_df.columns))
sql = f'INSERT INTO collision_incidents ({columns}) VALUES ({placeholders})'

# Convert DataFrame to list of tuples
data_to_insert = control_df.to_records(index=False)

# Execute the SQL command using executemany
cursor.executemany(sql, data_to_insert)

# Commit changes and close the connection
conn.commit()
conn.close()


print("Data successfully added to the SQLite database at", database_path)

In [80]:

# Write the DataFrame to the SQLite table
#combined_incidents_df.to_sql('collision_incidents', conn, if_exists='append', index=False)
combined_incidents_traffic_control_df.to_sql('incident_Traffic_control', conn, if_exists='append', index=False)
#vehicles_df.to_sql('incident_vehicles', conn, if_exists='append', index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()

print(f"Data from {df} has been successfully inserted into the collision_incidents table.")


Data from      IncidentId  TrafficControlNo       TrafficControl  Unnamed: 3
0      32658708                 1               MEDIAN         NaN
1      32659228                 1               MEDIAN         NaN
2      32672935                 1               MEDIAN         NaN
3      32676655                 1  ADVISORY SPEED SIGN         NaN
4      32676655                 2          CENTER LINE         NaN
..          ...               ...                  ...         ...
695    33431439                 3                OTHER         NaN
696    33431439                 4        WARNING SIGNS         NaN
697    33433119                 1               MEDIAN         NaN
698    33433119                 2  ADVISORY SPEED SIGN         NaN
699    33433119                 3        WARNING SIGNS         NaN

[700 rows x 4 columns] has been successfully inserted into the collision_incidents table.
